In [1]:
import requests
from datetime import datetime
import pandas as pd

firstUserID = 579
lastUserID = 973

pd.set_option("display.max_rows", 1000)

In [2]:
# 得到用户统计数据
def getUserStats(uid):
    r = requests.get('https://block7serv.heyalgo.io/v1/games/userstats?token=ewfbneivneghy3uryh&uid={}'.format(uid))
    return r.json()

# 解析时间字符串，2021-10-13_10:01:48
def parseTime(str):
    if str == '':
        return None
    
    return datetime.strptime(str, '%Y-%m-%d_%H:%M:%S')

# 计算时间差，返回整数小时差
def getTimeOffsetHours(timeNow, timeStart):
    if timeStart == None:
        return -1
    
    return int((timeNow.timestamp() - timeStart.timestamp()) / 60 / 60)

# dt = parseTime('2021-10-13_10:01:48')
# datetime.now()
# getTimeOffsetHours(datetime.now(), dt)

In [3]:
# 计算平均关卡星星，需要跳过5的倍数的关卡
def countAvgStageStars(levelArr):
    if levelArr == None:
        return 0
    
    totalstars = 0
    stagenums = 0
    
    for key in levelArr.keys():
        if int(key) % 5 != 0:
            totalstars = totalstars + int(levelArr[key])
            stagenums = stagenums + 1
    
    if stagenums > 0:
        return totalstars / stagenums
    
    return 0
        
# 计算下一关的流失率
def countNextStage(lststages, stage, curnums):
    if curnums > 0:
        for s in lststages:
            if s['stage'] == stage + 1:
                s['lostper'] = (curnums - s['totalusers']) / curnums
                
                if s['lostper'] < 0:
                    s['lostper'] = 0

                return
    else:
        s['lostper'] = 0
        
# 获取前一关的游戏人数       
def getPreStageTotalUsers(lststages, stage):
    for s in lststages:
        if s['stage'] == stage - 1:
            return s['totalusers']
            
    return 0

# 新增加一个玩家关卡统计
def addUserStages(lststages, stage, winper):
    for s in lststages:
        if s['stage'] == stage:
            s['totalwinper'] = s['totalwinper'] + winper
            s['totalusers'] = s['totalusers'] + 1
            s['winper'] = s['totalwinper'] / s['totalusers']
            
            ptnums = getPreStageTotalUsers(lststages, stage)
            if ptnums != 0:
                s['lostper'] = (ptnums - s['totalusers'])  / ptnums
                
                if s['lostper'] < 0:
                    s['lostper'] = 0
            else:
                s['lostper'] = 0
            
            countNextStage(lststages, stage, s['totalusers'])
            
            return
    
    ss = {
        'stage': stage,
        'totalwinper': winper,
        'totalusers': 1,
        'winper': winper,
        'totalClickNums': 0,
        'avgClickNums': 0,
        'totalAvgClickTime': 0,
        'avgClickTime': 0,
        'totalnums': 0,
        'totalWinClickNums': 0,
        'avgWinClickNums': 0,
        'totalWinAvgClickTime': 0,
        'avgWinClickTime': 0,
        'totalWinNums': 0,
        'totalLoseClickNums': 0,
        'avgLoseClickNums': 0,
        'totalLoseAvgClickTime': 0,
        'avgLoseClickTime': 0,
        'totalLoseNums': 0,        
    }
    
    lststages.append(ss)
    
    ss['lostper'] = (getPreStageTotalUsers(lststages, stage) - ss['totalusers']) / ss['totalusers']        
    countNextStage(lststages, stage, ss['totalusers'])


# 分析玩家历史数据
def analyzeUserStageHistory(ui, lststages, stage, stageHistory):
    for s in lststages:
        if s['stage'] == stage:
            s['totalnums'] = s['totalnums'] + 1
            
            s['totalClickNums'] = s['totalClickNums'] + stageHistory['clickNums']
            s['avgClickNums'] = s['totalClickNums'] / s['totalnums'] 
            
            s['totalAvgClickTime'] = s['totalAvgClickTime'] + stageHistory['avgClickTime']
            s['avgClickTime'] = s['totalAvgClickTime'] / s['totalnums']
            
            ui['totalHistoryNums'] = ui['totalHistoryNums'] + 1
            ui['totalAvgClickTime'] = ui['totalAvgClickTime'] + stageHistory['avgClickTime']
            ui['avgClickTime'] = ui['totalAvgClickTime'] / ui['totalHistoryNums']
            
            if stageHistory['gamestate'] == 1:
                s['totalWinNums'] = s['totalWinNums'] + 1

                s['totalWinClickNums'] = s['totalWinClickNums'] + stageHistory['clickNums']
                s['avgWinClickNums'] = s['totalWinClickNums'] / s['totalWinNums'] 

                s['totalWinAvgClickTime'] = s['totalWinAvgClickTime'] + stageHistory['avgClickTime']
                s['avgWinClickTime'] = s['totalWinAvgClickTime'] / s['totalWinNums']                
            else:
                s['totalLoseNums'] = s['totalLoseNums'] + 1

                s['totalLoseClickNums'] = s['totalLoseClickNums'] + stageHistory['clickNums']
                s['avgLoseClickNums'] = s['totalLoseClickNums'] / s['totalLoseNums'] 

                s['totalLoseAvgClickTime'] = s['totalLoseAvgClickTime'] + stageHistory['avgClickTime']
                s['avgLoseClickTime'] = s['totalLoseAvgClickTime'] / s['totalLoseNums']   
                
            return
            
    
# 分析玩家关卡数据    
def analyzeUserStages(ui, lststages, stages):
    if stages == None:
        ui['stagenums'] = 0
        
        return 
    
    nums = 0
    for key in stages.keys():
        if len(stages[key]['historys']) > 0:
            addUserStages(lststages, int(key), stages[key]['winnums'] / len(stages[key]['historys']))
            nums = nums + len(stages[key]['historys'])
            
            for sh in stages[key]['historys']:
                analyzeUserStageHistory(ui, lststages, int(key), sh)
                
            
    ui['stagenums'] = nums
        
    return
    

# 分析用户数据
def analyzeUserStats(startUID, endUID):
    lstui = []
    lststages = []
    timeNow = datetime.now()
    
    for uid in range(startUID, endUID + 1):
        cui = getUserStats(uid)
        ui = {
            'uid': uid, 
            'coin': int(cui['user']['coin']),
            'level': int(cui['user']['level']),
            'createTime': parseTime(cui['user']['createTime']),
            'lastTime': parseTime(cui['user']['lastLoginTime']),
            'avgStars': countAvgStageStars(cui['user']['levelarr']),
            'ip': cui['user']['ip'],
            'totalAvgClickTime': 0,
            'avgClickTime': 0,
            'totalHistoryNums': 0,
        }
        
        if ui['lastTime'] == None:
            ui['lastTime'] = ui['createTime']
        
        ui['offlineHours'] = getTimeOffsetHours(timeNow, ui['lastTime'])
        ui['aliveHours'] = getTimeOffsetHours(ui['lastTime'], ui['createTime'])
        
        analyzeUserStages(ui, lststages, cui['user']['stages'])
        
        lstui.append(ui)
    
    return lstui, lststages

In [4]:
lstui, lststages = analyzeUserStats(firstUserID, lastUserID)

In [24]:
df = pd.DataFrame(lstui)
df['createTime'] = pd.to_datetime(df['createTime'])

# df[(df['aliveHours'] > 0)&(df['level'] > 0)].sort_values("level")

In [26]:
#df[(df['createTime'] > datetime.strptime('2021-10-13', '%Y-%m-%d'))&(df['createTime'] < datetime.strptime('2021-10-14', '%Y-%m-%d'))]

In [27]:
df1 = pd.DataFrame(lststages)

# df1.sort_values("stage")